In [1]:
import PIL
import time
import torch
import torchvision
import torch.nn.functional as F
from einops import rearrange
from torch import nn
import torch.nn.init as init
from ViTResNet18 import *
from common import *
from TinyImageNet import TinyImageNet

In [2]:
PATH_TO_IMAGE_NET = "./data/tiny-imagenet-200"
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_VAL = 100
device = torch.device("cuda")

transform_train = torchvision.transforms.Compose(
     [torchvision.transforms.RandomHorizontalFlip(),
     torchvision.transforms.RandomRotation(10, resample=PIL.Image.BILINEAR),
     torchvision.transforms.RandomAffine(8, translate=(.15,.15)),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


transform_val = torchvision.transforms.Compose([
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = TinyImageNet(PATH_TO_IMAGE_NET, split='train', transform=transform_train, in_memory=False)
val_dataset = TinyImageNet(PATH_TO_IMAGE_NET, split='val', transform=transform_val, in_memory=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE_VAL, shuffle=False)

C:\Users\dmele\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\transforms\transforms.py:1200: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


In [3]:
for data, target in train_loader:
    print(data.shape)
    break

torch.Size([100, 3, 224, 224])


In [4]:
#model = ViTResNet(BasicBlock, [3, 3, 3], BATCH_SIZE_TRAIN, num_classes=200, num_tokens=16).to(device)



In [5]:
model = model = ViTResNet18(BasicBlock, [2, 2, 2], BATCH_SIZE_TRAIN, num_classes=200, num_tokens=16).to(device)
EPOCHS = 90
check_on_dataset(model, train_loader, val_loader, EPOCHS, "TinyImageNet", "ViTResNet18")

00 ( 80%)]  Loss: 0.4435
[90000/100000 ( 90%)]  Loss: 0.6797
Execution time: 245.46 seconds
Epoch: 43
[    0/100000 (  0%)]  Loss: 0.6392
[10000/100000 ( 10%)]  Loss: 0.5596
[20000/100000 ( 20%)]  Loss: 0.5215
[30000/100000 ( 30%)]  Loss: 0.5909
[40000/100000 ( 40%)]  Loss: 0.6886
[50000/100000 ( 50%)]  Loss: 0.4176
[60000/100000 ( 60%)]  Loss: 0.4725
[70000/100000 ( 70%)]  Loss: 0.6623
[80000/100000 ( 80%)]  Loss: 0.5163
[90000/100000 ( 90%)]  Loss: 0.6086
Execution time: 243.19 seconds
Epoch: 44
[    0/100000 (  0%)]  Loss: 0.5318
[10000/100000 ( 10%)]  Loss: 0.3706
[20000/100000 ( 20%)]  Loss: 0.3819
[30000/100000 ( 30%)]  Loss: 0.4872
[40000/100000 ( 40%)]  Loss: 0.5295
[50000/100000 ( 50%)]  Loss: 0.4642
[60000/100000 ( 60%)]  Loss: 0.3932
[70000/100000 ( 70%)]  Loss: 0.7066
[80000/100000 ( 80%)]  Loss: 0.7579
[90000/100000 ( 90%)]  Loss: 0.8311
Execution time: 242.06 seconds
Epoch: 45
[    0/100000 (  0%)]  Loss: 0.3185
[10000/100000 ( 10%)]  Loss: 0.5161
[20000/100000 ( 20%)]  L

In [12]:
from thop import profile
data, target = next(iter(val_loader))
flops, params = profile(model, inputs=(data.to(device), ))
print(data.shape)
print(f'flops = {flops}')
print(f'params = {params}')




[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'ViTResNet18.BasicBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'Tokenizers.FilterBasedTokenizer'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'Projector.Projector'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.normalization.LayerNorm'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear

In [3]:
model = ViTResNet18(BasicBlock, [2, 2, 2], BATCH_SIZE_TRAIN, num_classes=200,
                    num_tokens=16, tokenizers_type=['pooling', 'pooling']).to(device)
EPOCHS = 90
check_on_dataset(model, train_loader, val_loader, EPOCHS, "TinyImageNet", "ViTResNet18(pooling)")

00 ( 80%)]  Loss: 0.7422
[90000/100000 ( 90%)]  Loss: 0.5931
Execution time: 248.05 seconds
Epoch: 43
[    0/100000 (  0%)]  Loss: 0.7177
[10000/100000 ( 10%)]  Loss: 0.8898
[20000/100000 ( 20%)]  Loss: 0.6926
[30000/100000 ( 30%)]  Loss: 0.5669
[40000/100000 ( 40%)]  Loss: 0.6249
[50000/100000 ( 50%)]  Loss: 0.5604
[60000/100000 ( 60%)]  Loss: 0.5457
[70000/100000 ( 70%)]  Loss: 0.6415
[80000/100000 ( 80%)]  Loss: 0.7039
[90000/100000 ( 90%)]  Loss: 0.5461
Execution time: 249.42 seconds
Epoch: 44
[    0/100000 (  0%)]  Loss: 0.6287
[10000/100000 ( 10%)]  Loss: 0.6565
[20000/100000 ( 20%)]  Loss: 0.5972
[30000/100000 ( 30%)]  Loss: 0.6109
[40000/100000 ( 40%)]  Loss: 0.6395
[50000/100000 ( 50%)]  Loss: 0.5921
[60000/100000 ( 60%)]  Loss: 0.5124
[70000/100000 ( 70%)]  Loss: 0.5551
[80000/100000 ( 80%)]  Loss: 0.4657
[90000/100000 ( 90%)]  Loss: 0.7325
Execution time: 249.34 seconds
Epoch: 45
[    0/100000 (  0%)]  Loss: 0.3857
[10000/100000 ( 10%)]  Loss: 0.6644
[20000/100000 ( 20%)]  L

In [2]:
PATH_TO_IMAGE_NET = "./data/tiny-imagenet-201"
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_VAL = 100
device = torch.device("cuda")

transform_train = torchvision.transforms.Compose(
     [torchvision.transforms.RandomHorizontalFlip(),
     torchvision.transforms.RandomRotation(10, resample=PIL.Image.BILINEAR),
     torchvision.transforms.RandomAffine(8, translate=(.15,.15)),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


transform_val = torchvision.transforms.Compose([
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = TinyImageNet(PATH_TO_IMAGE_NET, split='train', transform=transform_train, in_memory=False)
val_dataset = TinyImageNet(PATH_TO_IMAGE_NET, split='val', transform=transform_val, in_memory=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE_VAL, shuffle=False)

C:\Users\dmele\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\transforms\transforms.py:1200: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  warnings.warn(


In [3]:
for data, target in train_loader:
    print(data.shape)
    break


torch.Size([100, 3, 64, 64])


In [4]:
model = model = ViTResNet18(BasicBlock, [2, 2, 2], BATCH_SIZE_TRAIN, num_classes=200, num_tokens=16).to(device)
EPOCHS = 90
check_on_dataset(model, train_loader, val_loader, EPOCHS, "TinyImageNet(64x64)", "ViTResNet18")

0
[70000/100000 ( 70%)]  Loss: 1.3834
[80000/100000 ( 80%)]  Loss: 1.0729
[90000/100000 ( 90%)]  Loss: 1.2192
Execution time: 63.81 seconds
Epoch: 43
[    0/100000 (  0%)]  Loss: 1.1735
[10000/100000 ( 10%)]  Loss: 1.7379
[20000/100000 ( 20%)]  Loss: 1.5670
[30000/100000 ( 30%)]  Loss: 1.3198
[40000/100000 ( 40%)]  Loss: 1.3900
[50000/100000 ( 50%)]  Loss: 1.2089
[60000/100000 ( 60%)]  Loss: 1.1863
[70000/100000 ( 70%)]  Loss: 1.3198
[80000/100000 ( 80%)]  Loss: 1.6121
[90000/100000 ( 90%)]  Loss: 1.4708
Execution time: 63.72 seconds
Epoch: 44
[    0/100000 (  0%)]  Loss: 1.1013
[10000/100000 ( 10%)]  Loss: 1.2034
[20000/100000 ( 20%)]  Loss: 1.4594
[30000/100000 ( 30%)]  Loss: 0.9215
[40000/100000 ( 40%)]  Loss: 1.3133
[50000/100000 ( 50%)]  Loss: 1.0064
[60000/100000 ( 60%)]  Loss: 1.0334
[70000/100000 ( 70%)]  Loss: 1.0345
[80000/100000 ( 80%)]  Loss: 1.2710
[90000/100000 ( 90%)]  Loss: 1.4359
Execution time: 63.88 seconds
Epoch: 45
[    0/100000 (  0%)]  Loss: 1.2017
[10000/100000 

In [3]:
PATH_TO_IMAGE_NET = "./data/tiny-imagenet-200"
BATCH_SIZE_TRAIN = 100
BATCH_SIZE_VAL = 100
device = torch.device("cuda")

transform_train = torchvision.transforms.Compose(
     [torchvision.transforms.RandomHorizontalFlip(),
     torchvision.transforms.RandomRotation(10, resample=PIL.Image.BILINEAR),
     torchvision.transforms.RandomAffine(8, translate=(.15,.15)),
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


transform_val = torchvision.transforms.Compose([
     torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = TinyImageNet(PATH_TO_IMAGE_NET, split='train', transform=transform_train, in_memory=False)
val_dataset = TinyImageNet(PATH_TO_IMAGE_NET, split='val', transform=transform_val, in_memory=False)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE_VAL, shuffle=False)

In [4]:
model = ViTResNet18(BasicBlock, [2, 2, 2], BATCH_SIZE_TRAIN, num_classes=200,
                    num_tokens=16, tokenizers_type=['filter', 'recurrent']).to(device)
EPOCHS = 90
check_on_dataset(model, train_loader, val_loader, EPOCHS, "TinyImageNet", "ViTResNet18(recurrent)")

Loaded model's checkpoint
Epoch: 31
[    0/100000 (  0%)]  Loss: 1.2030
[10000/100000 ( 10%)]  Loss: 0.8573
[20000/100000 ( 20%)]  Loss: 0.9443
[30000/100000 ( 30%)]  Loss: 0.8558
[40000/100000 ( 40%)]  Loss: 0.8516
[50000/100000 ( 50%)]  Loss: 0.7787
[60000/100000 ( 60%)]  Loss: 1.0315
[70000/100000 ( 70%)]  Loss: 0.7017
[80000/100000 ( 80%)]  Loss: 0.6137
[90000/100000 ( 90%)]  Loss: 0.8249
Execution time: 249.51 seconds
Epoch: 32
[    0/100000 (  0%)]  Loss: 0.6816
[10000/100000 ( 10%)]  Loss: 0.7427
[20000/100000 ( 20%)]  Loss: 1.0007
[30000/100000 ( 30%)]  Loss: 0.7531
[40000/100000 ( 40%)]  Loss: 0.7558
[50000/100000 ( 50%)]  Loss: 0.9326
[60000/100000 ( 60%)]  Loss: 0.9505
[70000/100000 ( 70%)]  Loss: 0.7547
[80000/100000 ( 80%)]  Loss: 0.8900
[90000/100000 ( 90%)]  Loss: 0.6430
Execution time: 248.97 seconds
Epoch: 33
[    0/100000 (  0%)]  Loss: 0.9509
[10000/100000 ( 10%)]  Loss: 0.5628
[20000/100000 ( 20%)]  Loss: 0.8402


KeyboardInterrupt: 